In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [ ]:
df = pd.read_csv('/content/Supermarket_sales_prediction.csv')


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [ ]:
df.isnull().sum()

,0
Item_Identifier,0
Item_Weight,1463
Item_Fat_Content,0
Item_Visibility,0
Item_Type,0
Item_MRP,0
Outlet_Identifier,0
Outlet_Establishment_Year,0
Outlet_Size,2410
Outlet_Location_Type,0


In [ ]:
df['Outlet_Size'].fillna(df['Outlet_Size'].mode()[0],inplace=True)
df['Item_Weight'].fillna(df['Item_Weight'].median(),inplace=True)

<ipython-input-39-32ca924537ea>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Outlet_Size'].fillna(df['Outlet_Size'].mode()[0],inplace=True)
<ipython-input-39-32ca924537ea>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplac

In [ ]:
df.duplicated().sum()

0

In [ ]:
encoder = LabelEncoder()
for col in df.select_dtypes(include=['object']).columns:
  encoder.fit(df[col].unique())
  df[col] = encoder.transform(df[col])

In [ ]:
pca = PCA(n_components=5)
x_pca = pca.fit_transform(df)
df_pca = pd.DataFrame(data=x_pca,columns=['PC1','PC2','PC3','PC4','PC5'])
df_pca['Item_Outlet_Sales'] = df['Item_Outlet_Sales']
df_pca

,PC1,PC2,PC3,PC4,PC5,Item_Outlet_Sales
0,1555.263148,-624.860391,77.542972,1.003590,-4.971416,3735.1380
1,-1740.044645,-770.394220,-55.390087,10.843215,-0.318121,443.4228
2,-84.082626,-117.641614,2.550916,1.296811,5.337919,2097.2700
3,-1447.468850,342.577625,70.618265,-0.691410,3.767284,732.3800
4,-1187.711767,518.153344,-63.391029,-10.729373,-1.496312,994.7052
...,...,...,...,...,...,...
8518,598.158370,-410.105437,61.732906,-11.355113,-0.650090,2778.3834
8519,-1632.238631,118.612838,0.794007,3.841687,-8.109138,549.2850
8520,-988.652167,578.030663,-36.256063,6.357776,-1.144948,1193.1136
8521,-336.486935,-98.495608,-30.654419,11.169399,-0.291330,1845.5976


In [ ]:
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_pca)
df_scaled = pd.DataFrame(df_scaled,columns=['PC1','PC2','PC3','PC4','PC5','Item_Outlet_Sales'])
df_scaled

,PC1,PC2,PC3,PC4,PC5,Item_Outlet_Sales
0,0.911234,-1.391068,1.512845,0.120052,-1.163299,0.910601
1,-1.019498,-1.715057,-1.080647,1.297088,-0.074440,-1.018440
2,-0.049264,-0.261895,0.049768,0.155127,1.249060,-0.049238
3,-0.848077,0.762649,1.377745,-0.082708,0.881535,-0.849103
4,-0.695884,1.153516,-1.236744,-1.283470,-0.350133,-0.695373
...,...,...,...,...,...,...
8518,0.350463,-0.912979,1.204394,-1.358323,-0.152120,0.349915
8519,-0.956334,0.264057,0.015491,0.459551,-1.897518,-0.956402
8520,-0.579254,1.286816,-0.707347,0.760531,-0.267915,-0.579100
8521,-0.197149,-0.219272,-0.598060,1.336107,-0.068170,-0.196725


In [ ]:
x = df_scaled.drop('Item_Outlet_Sales',axis=1)
y = df_scaled['Item_Outlet_Sales']
x_pca_train,x_pca_test,y_pca_train,y_pca_test=train_test_split(x,y,test_size=0.2,random_state=2)

In [ ]:
model1 = LinearRegression()
model1.fit(x_pca_train,y_pca_train)
y_pca_predict1=model1.predict(x_pca_test)
print("R2:",r2_score(y_pca_test,y_pca_predict1))

R2: 0.9999999999996103


In [ ]:
model2 = RandomForestRegressor()
model2.fit(x_pca_train,y_pca_train)
y_pca_predict2=model2.predict(x_pca_test)
print("R2:",r2_score(y_pca_test,y_pca_predict2))

R2: 0.9999703392667124


In [ ]:
model3 = GradientBoostingRegressor()
model3.fit(x_pca_train,y_pca_train)
y_pca_predict3 = model3.predict(x_pca_test)
print("R2:",r2_score(y_pca_test,y_pca_predict3))

R2: 0.9999026098197102


In [ ]:
model4 = lgb.LGBMRegressor(boosting_type='gbdt', objective='regression',
                           n_estimators=100, learning_rate=0.1, random_state=42)
model4.fit(x_pca_train, y_pca_train)
y_pca_predict4 = model4.predict(x_pca_test)
print("R2 Score:",r2_score(y_pca_test,y_pca_predict4))

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000419 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1275
[LightGBM] [Info] Number of data points in the train set: 6818, number of used features: 5
[LightGBM] [Info] Start training from score -0.008032
R2 Score: 0.9987549522624907
